Setup and Initial Auth

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


Import ADK components

In [2]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents.callback_context import CallbackContext
from google.adk.plugins.base_plugin import BasePlugin

# Built-in Google tool (Gemini 2 compatible)
from google.adk.tools import google_search,AgentTool

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

Python imports

In [4]:
import argparse
import asyncio
import logging
import math
import os
import time
from dataclasses import dataclass
from typing import Any, Dict, Optional

**Simple Calc tool (custom tool for agent)**
Custom tool to do basic math functions
calc Tool: lightweight numeric helper for quantitative claims.

In [5]:
def calc(expression: str) -> Dict[str, Any]:
    """
    Safe(ish) mini math evaluator: supports Python 'math' only.
    Examples: "2*(3+4)", "math.log(8, 2)"
    """
    allowed = {"__builtins__": {}, "math": math}
    try:
        result = eval(expression, allowed, {})
        return {"status": "success", "result": result}
    except Exception as e:
        return {"status": "error", "error": str(e)}

**Models**
we are going to use Gemini models for this

In [6]:
RESEARCH_MODEL  = "gemini-2.5-flash"
DEBATE_MODEL    = "gemini-2.5-flash"
TUTOR_MODEL     = "gemini-2.0-flash"
ROOT_MODEL      = "gemini-2.5-flash"

**Observability (Using Logging Plugin)**
A Plugin in Agent Development Kit (ADK) is a custom code module that can be executed at various stages of an agent workflow lifecycle using callback hooks. You use Plugins for functionality that is applicable across your agent workflow.
An ADK Plugin extends the BasePlugin class and contains one or more callback methods, indicating where in the agent lifecycle the Plugin should be executed. You integrate Plugins into an agent by registering them in your agent's Runner class.
Logging: Log important information at each agent workflow callback point.

In [7]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
) 


**Agents (5): Research,Pro, Con, Scribe-Tutor, Moderator(root)**
* Root Agent (Gemini 2.5-Flash): coordinates workflow and calls each sub-agent as a tool.
* Research Agent (Gemini 2.5-Flash): performs Google Search to gather 5-7 authoritative sources.
* Pro / Con Agents (Gemini 2.5-Flash): generate structured arguments with citations [src_k].
* Scribe-Tutor Agent (Gemini 2.0-Flash): summarizes debate into notes, glossary, and quiz.

**Sessions & State (InMemoryRunner)**
Maintains context automatically per run


In [8]:
COMMON_SUFFIX = (
    "Use the google_search tool to find recent or authoritative sources when needed.\n"
    "Include citation tags like [src_k] (e.g., [src_1]) near each claim.\n"
    "Prefer primary sources, official reports, or reputable orgs. Avoid fallacies; quantify when helpful.\n"
)
research_agent = LlmAgent(
    name="research_agent",
    model=Gemini(model=RESEARCH_MODEL,retry_options=retry_config),
    description="Finds and condenses authoritative recent sources for a topic.",
    instruction=(
        "Use google_search to find 5–7 recent, authoritative sources on the topic. "
        "Prefer primary/official publications (.gov/.edu/.org), reputable journals, or major technical blogs. "
        "Return a numbered list [src_1]..[src_k] with:\n"
        "• Title\n• URL\n• 1–2 sentence takeaway (why it matters)\n"
        "Avoid duplicates; ensure recency where relevant."
    ),
    tools=[google_search],
)
# 2) Pro agent — argues FOR (no web tools needed, cites [src_k] from research)
pro_agent = LlmAgent(
    name="pro_agent",
    model=Gemini(model=DEBATE_MODEL,retry_options=retry_config),
    description="Argues FOR the claim with structured reasoning and citations.",
    instruction=(
        "Produce EXACTLY 3 numbered claims FOR the claim.\n"
        "For each claim: 1–2 sentence justification. Cite ONLY [src_k] provided by the research step.\n"
        "Quantify when helpful; avoid fallacies."
    ),
    tools=[calc],
)

# 3) Con agent — argues AGAINST
con_agent = LlmAgent(
    name="con_agent",
    model=Gemini(model=DEBATE_MODEL,retry_options=retry_config),
    description="Argues AGAINST the claim with structured reasoning and citations.",
    instruction=(
        "Produce EXACTLY 3 numbered counter-claims AGAINST the claim.\n"
        "For each claim: 1–2 sentence justification. Cite ONLY [src_k] provided by the research step.\n"
        "Quantify when helpful; avoid fallacies."
    ),
    tools=[calc],
)

# 4) Scribe-Tutor — converts debate into learning pack
scribe_tutor = LlmAgent(
    name="scribe_tutor",
    model=Gemini(model=TUTOR_MODEL,retry_options=retry_config),
    description="Converts debate transcript and sources into student-ready materials.",
    instruction=(
        "Create a concise learning pack based on the debate & sources:\n"
        "1) Pros/Cons table (Point | Impact | Evidence strength)\n"
        "2) 5 key takeaways\n"
        "3) 5-term glossary (term: short definition)\n"
        "4) 5-question quiz (MCQ or short answer) with 1-sentence rationale each.\n"
        "Keep the reading level aligned to the user's level if provided."
    ),
)

# =============================================================================
# Root agent — orchestrates via AgentTool(...) calls
# =============================================================================

root_agent = LlmAgent(
    name="edudebate_root",
    model=Gemini(model=ROOT_MODEL,retry_options=retry_config),
    description="Root coordinator for research→debate→tutor educational flow.",
    instruction=(
        "You orchestrate an educational debate using callable tools:\n"
        "1) Call Research Agent to get 5–7 sources on the given topic. Keep the returned list as [src_k].\n"
        "2) Call Pro Agent to produce exactly 3 FOR claims that cite [src_k].\n"
        "3) Call Con Agent to produce exactly 3 AGAINST claims that cite [src_k].\n"
        "4) Ask each side for a brief 120–150 word closer summarizing their strongest points with [src_k].\n"
        "5) Call Scribe-Tutor to generate the learning pack (notes, pros/cons, glossary, quiz).\n"
        "Return a clean final answer with: the sources, both sides' closers, and the learning pack."
    ),
    tools=[
        AgentTool(research_agent),
        AgentTool(pro_agent),
        AgentTool(con_agent),
        AgentTool(scribe_tutor),
        calc,  # optional numeric helper for root if needed
    ],
)
# Runner with standardized observability
runner = InMemoryRunner(agent=root_agent, plugins=[LoggingPlugin()])


Orchestration (With Runner and InMemorySessionService)

In [9]:
async def main(topic: str, level: str = "beginner"):
    """
    One-shot debug run. The root agent will call Research→Pro→Con→Tutor via AgentTool.
    """
    prompt = f"Topic: {topic}\nLevel: {level}\n\nPlease follow the orchestration steps."
    resp = await runner.run_debug(prompt)

    # ADK versions may return a single object or list; try best-effort printing
    try:
        print("\n🎓 EduDebate AI — OUTPUT\n" + "="*80)
        print(getattr(resp, "output_text", "") or str(resp))
    except Exception:
        # Fallback: if it's a list, print each item
        if isinstance(resp, list):
            print("\n🎓 EduDebate AI — OUTPUT (list)\n" + "="*80)
            for i, r in enumerate(resp, 1):
                print(f"\n--- PART {i} ---\n")
                print(getattr(r, "output_text", "") or str(r))
        else:
            print(resp)

# Optional script entry (if run as a .py outside notebooks)
if __name__ == "__main__":
    import asyncio
    try:
        asyncio.get_running_loop()
        print("⚠️ Detected running event loop. In notebooks, call: await main(...).")
    except RuntimeError:
        asyncio.run(main("What are the latest advancements in quantum computing and what do they mean for AI?", "intermediate"))

⚠️ Detected running event loop. In notebooks, call: await main(...).


In [10]:
# In a Kaggle/Jupyter cell after the file is loaded/run:
await main(
    "What are the latest advancements in quantum computing and what do they mean for AI?",
    level="intermediate"
)


 ### Created new session: debug_session_id

User > Topic: What are the latest advancements in quantum computing and what do they mean for AI?
Level: intermediate

Please follow the orchestration steps.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-5ba7fee2-b948-4a54-be1b-bd6979956707
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: edudebate_root
[logging_plugin]    User Content: text: 'Topic: What are the latest advancements in quantum computing and what do they mean for AI?
Level: intermediate

Please follow the orchestration steps.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-5ba7fee2-b948-4a54-be1b-bd6979956707
[logging_plugin]    Starting Agent: edudebate_root
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: edudebate_root
[logging_plugin]    Invocation ID: e-5ba7fee2-b948-4a54-be1b

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: research_agent
[logging_plugin]    Token Usage - Input: 466, Output: 24
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 9b23c8f4-71a1-4b05-b997-63754c0d248f
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: research_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['research_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: research_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-cd93943d-b4ee-4c1c-b4e0-47e332641c71
[logging_plugin]    Arguments: {'request': 'Latest advancements in quantum computing and their implications for AI'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-a2f56aa7-e7c7-4adb-876d-bc21a5138f0e
[logging_plugin]    Session ID: 746d59f9-0720-4c6e-8163-535ef961dc35
[logging_p

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: text: 'Here are the sources for our debate:

1.  **New algorithm unlocks the power of quantum machine learning - Advanced Science News**
    *   URL: [https://vertexaisearch.cloud.google.com/grounding-api-re...' | function_call: pro_agent
[logging_plugin]    Token Usage - Input: 3646, Output: 3311
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: fd6ab78e-6f1b-40a2-a95a-bf58da884e56
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: text: 'Here are the sources for our debate:

1.  **New algorithm unlocks the power of quantum machine learning - Advanced Science News**
    *   URL: [https://vertexaisearch.cloud.google.com/grounding-api-re...' | function_call: pro_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['pro_agent']
edudebate_root > Here are the sources for our debate:

1.  **New algorithm unlocks the power of

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: pro_agent
[logging_plugin]    Content: text: 'Here are 3 claims supporting the impact of quantum computing advancements on AI:

1.  Quantum algorithms are unlocking new capabilities for machine learning. A new algorithm has been developed to unlo...'
[logging_plugin]    Token Usage - Input: 318, Output: 239
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 3c5720e4-5ce6-45bf-9e33-9e09c3499751
[logging_plugin]    Author: pro_agent
[logging_plugin]    Content: text: 'Here are 3 claims supporting the impact of quantum computing advancements on AI:

1.  Quantum algorithms are unlocking new capabilities for machine learning. A new algorithm has been developed to unlo...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: pro_agent
[logging_plugin]    Invocation ID: e-14d2c80b-7f38-4112-9a17-e37016382147
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    I

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: con_agent
[logging_plugin]    Token Usage - Input: 7214, Output: 187
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: f0fe8882-db0d-4d5c-8f3c-2b14dde80447
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: con_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['con_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: con_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-0a2e2220-f8cd-4d87-8112-f38f13f9ff57
[logging_plugin]    Arguments: {'request': 'Produce exactly 3 AGAINST claims about the latest advancements in quantum computing and what they mean for AI, citing the following sources:\n1. Advanced Science News: "New algorithm unlocks the power of quantum machine learning"\n2. Your Quick Guide to Quantum and AI: The Future of C

...

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: con_agent
[logging_plugin]    Content: text: '1.  **Current quantum systems lack fault tolerance and a proven, practical "quantum advantage" for most AI applications.** While new algorithms are emerging [src_1], the path to achieving reliable fau...'
[logging_plugin]    Token Usage - Input: 352, Output: 343
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 50840402-bfd5-45f9-8d7e-6bb6e4e451d9
[logging_plugin]    Author: con_agent
[logging_plugin]    Content: text: '1.  **Current quantum systems lack fault tolerance and a proven, practical "quantum advantage" for most AI applications.** While new algorithms are emerging [src_1], the path to achieving reliable fau...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: con_agent
[logging_plugin]    Invocation ID: e-b67e11eb-509b-406b-ba6f-e8bdc6bd49de
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    I

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: text: 'My apologies. The con-agent's response still included numbered points before the paragraph, and the paragraph itself was quite long. I need to re-emphasize the requirement for a *single paragraph* wit...' | function_call: con_agent
[logging_plugin]    Token Usage - Input: 9783, Output: 292
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 8e5b41f8-7a27-4126-9236-d70fdcbfff06
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: text: 'My apologies. The con-agent's response still included numbered points before the paragraph, and the paragraph itself was quite long. I need to re-emphasize the requirement for a *single paragraph* wit...' | function_call: con_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['con_agent']
edudebate_root > My apologies. The con-agent's response still included numbered points before th

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: con_agent
[logging_plugin]    Content: text: 'Despite promising algorithmic advancements in quantum machine learning [src_1], the immediate revolutionary impact of quantum computing on AI remains highly uncertain, with ongoing debate questioning ...'
[logging_plugin]    Token Usage - Input: 365, Output: 133
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 01e58e14-3e4b-4f2a-a17a-914604c5c1fd
[logging_plugin]    Author: con_agent
[logging_plugin]    Content: text: 'Despite promising algorithmic advancements in quantum machine learning [src_1], the immediate revolutionary impact of quantum computing on AI remains highly uncertain, with ongoing debate questioning ...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: con_agent
[logging_plugin]    Invocation ID: e-bc9fab9f-2a7f-48db-8c7b-41f5fa2e53f3
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    I

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: text: 'Here are the closers from both sides:

**Pro Closer:**
Quantum computing is rapidly advancing, poised to revolutionize AI through groundbreaking developments. New algorithms are now unlocking the full...' | function_call: scribe_tutor
[logging_plugin]    Token Usage - Input: 10222, Output: 2596
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 7a04ede0-b46d-4b12-9e5c-6e936b06d1cc
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: text: 'Here are the closers from both sides:

**Pro Closer:**
Quantum computing is rapidly advancing, poised to revolutionize AI through groundbreaking developments. New algorithms are now unlocking the full...' | function_call: scribe_tutor
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['scribe_tutor']
edudebate_root > Here are the closers from both sides:

**Pro Closer:**
Quantum compu

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: text: 'Here is the comprehensive educational debate summary on the latest advancements in quantum computing and their implications for AI:

### Sources:

1.  **New algorithm unlocks the power of quantum mach...'
[logging_plugin]    Token Usage - Input: 14196, Output: 4664
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: a9b0795a-c642-4400-8702-6a7b37afcc24
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: text: 'Here is the comprehensive educational debate summary on the latest advancements in quantum computing and their implications for AI:

### Sources:

1.  **New algorithm unlocks the power of quantum mach...'
[logging_plugin]    Final Response: True
edudebate_root > Here is the comprehensive educational debate summary on the latest advancements in quantum computing and their implications for AI:

### Sources:

1.  **New algorithm unlocks the pow